# Bigram POS tagger with hidden Markov model and Viterbi algorithm.

In [1]:
import nltk
import re
from conllu import parse
from nltk.corpus import conll2000
from collections import defaultdict
from collections import Counter
conll2000.ensure_loaded()

In [2]:
with open('UD_English-EWT-master/en_ewt-ud-train.conllu') as train:
    train_sents = []
    for tokenlist in parse(train.read()):
        train_sents.append([(token['form'], token['upostag']) for token in tokenlist])
with open('UD_English-EWT-master/en_ewt-ud-test.conllu') as test:
    test_sents = []
    for tokenlist in parse(test.read()):
        test_sents.append([(token['form'], token['upostag']) for token in tokenlist])

In [3]:
train_sents_conll2000 = conll2000.tagged_sents()[:8000]
test_sents_conll2000 = conll2000.tagged_sents()[8000:]

In [4]:
def accuracy(test_sents, postagger):
    errors = 0
    length = 0
    for sent in test_sents:
        length += len(sent)
        sent, real_tags = zip(*sent)
        my_tags = postagger.tag(sent)
        for i in range(len(my_tags)):
            if my_tags[i][1] != real_tags[i]:
                errors += 1
    return 1 - errors / length

In [5]:
# Нормализатор для получения распределения вероятностей из частот
class BaseNormalizer:
    def normalize(self, counter):
        sum_ = sum(counter.values())
        return {token: (counter[token] / sum_) for token in counter}

Для каждого слова будем выбирать наиболее вероятный тег, учитывая общую вероятность самого тега вместе с тегом предыдущего слова.
$$
        tag(w) = \arg \max_{i \in 1 .. |Tags| } P(w \mid tag_i)P(tag_i \mid t_{i - 1})
$$

In [6]:
class EmissionModel: #остаётся такой же, как и в униграммном тэггере
    def __init__(self, tagged_sents, normalizer=BaseNormalizer()):
        self.normalizer = normalizer
        self.model = defaultdict(Counter)
        # self.model будет иметь вид 
        # defaultdict({'tag_1': Counter({'word_1': 0.3, 'word_2': 0.7}), 'tag_2': Counter({'word_1': 0.6, 'word_3': 0.3 ...})})
        for sent in tagged_sents:
            for word, tag in sent:
                self.model[tag][word] += 1
        self.add_unk_token()
        for tag in self.model:
            self.normalizer.normalize(self.model[tag])
        
    def add_unk_token(self):
        # Для каждого тега добавим одинаковую вероятность быть приписанным любому слову, которого нет в модели
        for tag in self.model:
            self.model[tag]['UNK'] = 0.1
        
    def tags(self):
        # Добавим возможность возвращать все теги, которые есть в модели
        return self.model.keys()
    
    def __getitem__(self, tag):
        # Все слова для данного тега
        return self.model[tag]
    
    def __call__(self, word, tag):
        # Самое интересное - вероятность P(word|tag)
        if word not in self[tag]:
            return self[tag]['UNK']
        return self[tag][word]

In [7]:
class TransitionModel:
    def __init__(self, tag_seqs, normalizer=BaseNormalizer()):
        self.normalizer = normalizer
        # Эта модель будет иметь вид:
        # defaultdict({'tag_1': Counter({'tag_1': 0.3, 'tag_2': 0.7}), 'tag_2': Counter({'tag_1': 0.6, 'tag_3': 0.3 ...})})
        self.model = defaultdict(Counter)
        for seq in tag_seqs:
            self.model[None][seq[0]] += 1 #вероятности быть встреченным в начале слова будем хранить в self.model[None]
            for i in range(1, len(seq)):
                # проходимся по каждому тэгу последовательности, начиная со второго, для каждого записываем
                # каждое вхождение после предыдущего тэга в self.model
                self.model[seq[i - 1]][seq[i]] += 1
        for tag in self.model:
            self.normalizer.normalize(self.model[tag])
        
    def tags(self):
        return self.model.keys()
    
    def __getitem__(self, tag):
        # все теги, которые встречаются перед данным тегом
        return self.model[tag]
    
    def __call__(self, tag, prev_tag):
        # вероятность P(tag|prev_tag)
        return self.model[prev_tag][tag]

In [8]:
class BigramPOSTagger:
    def __init__(self, emission_model, transition_model):
        self.em = emission_model
        self.tm = transition_model
    
    def tag(self, sent):
        # Для списка слов возвращаем список пар (слово, тег)
        # Для каждого слова проходимся по всем тегам
        # И выбираем максимум по формуле
        tags = []
        for i in range(len(sent)):
            word = sent[i]
            if i == 0:
                prev_t = None
            else:
                prev_t = tags[i - 1]
            max_prob = 0
            best_tag = 'UNK'
            for t in self.tm.tags():
                prob = self.em(word, t) * self.tm(t, prev_t)
                if prob > max_prob:
                    max_prob, best_tag = prob, t
            tags.append(best_tag)
        return list(zip(sent, tags))

In [9]:
# Проверяем!
em = EmissionModel(train_sents)
tm = TransitionModel([[tag for word, tag in sent] for sent in train_sents])
bigram_postagger = BigramPOSTagger(em, tm) #теггер, обученный на UD
em_conll2000 = EmissionModel(train_sents_conll2000)
tm_conll2000 = TransitionModel([[tag for word, tag in sent] for sent in train_sents_conll2000])
bigram_postagger_conll2000 = BigramPOSTagger(em_conll2000, tm_conll2000) #и на conll2000

In [10]:
print('Точность тэггера, обученного на корпусе UD: {}'.format(accuracy(test_sents, bigram_postagger)))
print('Точность тэггера, обученного на корпусе conll2000: {}'.format(accuracy(test_sents_conll2000, bigram_postagger_conll2000)))

Точность тэггера, обученного на корпусе UD: 0.8309359684424433
Точность тэггера, обученного на корпусе conll2000: 0.91444050603729


Accuracy на conll2000 лучше!

In [11]:
bigram_postagger.tag('The quick brown fox jumps over the lazy dog'.split(' ')) #проверяем!

[('The', 'DET'),
 ('quick', 'ADJ'),
 ('brown', 'NOUN'),
 ('fox', 'PUNCT'),
 ('jumps', 'VERB'),
 ('over', 'ADP'),
 ('the', 'DET'),
 ('lazy', 'ADJ'),
 ('dog', 'NOUN')]

In [12]:
bigram_postagger_conll2000.tag('The quick brown fox jumps over the lazy dog'.split(' '))

[('The', 'DT'),
 ('quick', 'JJ'),
 ('brown', 'NN'),
 ('fox', 'NN'),
 ('jumps', 'IN'),
 ('over', 'IN'),
 ('the', 'DT'),
 ('lazy', 'NN'),
 ('dog', 'NN')]